# Notebook Contoso

###### Notebook criado para adicionar colunas em português das dimensões do dataset "Contoso" com o objetivo de gerar um modelo semântico do Power BI em português.

###### Este código utiliza a API do Azure Translator para gerar os dados das dimensões, além de fazer algumas traduções de acordo com valores pré-definidos antes de chamar a API.

### Coletar dados do Github

###### Lê os arquivos CSV do dataset "Contoso" do Github e carrega para o lakehouse padrão "LH_Contoso"

In [ ]:
import requests

# Lista de arquivos no GitHub (raw URLs)
arquivos = [
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/currencyexchange.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/customer.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/date.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/orderrows.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/orders.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/product.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/sales.csv",
    "https://raw.githubusercontent.com/mvbfontes/DatasetsTreinamento/refs/heads/main/Contoso/store.csv",
]

for url in arquivos:
    nome = url.split("/")[-1]  # pega o nome do arquivo
    destino = f"/lakehouse/default/Files/{nome}"
    
    response = requests.get(url)
    response.raise_for_status()
    
    with open(destino, "wb") as f:
        f.write(response.content)
    
    print(f"{nome} copiado para {destino}")


### Copiando arquivo da pasta "Files" para "Tables"

###### Gera tabelas no lakehouse padrão "LH_Contoso" com o prefixo "_stg" a partir dos arquivos carregados na pasta "Files"

In [ ]:
# Lista de arquivos que você copiou para a pasta Files
arquivos = ["currencyexchange.csv", "customer.csv", "date.csv", "orderrows.csv", "orders.csv", "product.csv", "sales.csv", "store.csv"]

for nome in arquivos:
    caminho = f"Files/{nome}"
    tabela = f"stg_{nome.split('.')[0]}"

    df = (
        spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .option("delimiter", ",")
        .load(caminho)
    )

    df.write.format("delta").mode("overwrite").saveAsTable(tabela)
    print(f"Tabela {tabela} criada com sucesso!")


### Criar colunas traduzidas para português - tabela Customer

###### Cria a tabela "silver_customer" adicionando novas colunas com tradução em português (gênero, ocupação, país e continente)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import requests
import json

# ============= CONFIGURE AQUI =============
TRANSLATOR_KEY = "xxx"
TRANSLATOR_ENDPOINT = "https://api.cognitive.microsofttranslator.com"
TRANSLATOR_LOCATION = "xxx"
# ==========================================

# Dicionário de traduções personalizadas
CUSTOM_TRANSLATIONS = {
    # Gêneros
    "male": "Masculino",
    "female": "Feminino",
    
    # Continentes
    "North America": "América do Norte",
    "South America": "América do Sul",
    "Central America": "América Central",
    "Europe": "Europa",
    "Asia": "Ásia",
    "Africa": "África",
    "Oceania": "Oceania",
    
    # Países
    "United States": "Estados Unidos",
    "United Kingdom": "Reino Unido",
    "Brazil": "Brasil",
    "Germany": "Alemanha",
    "France": "França",
    "Spain": "Espanha",
    "Italy": "Itália",
    "Portugal": "Portugal",
    "China": "China",
    "Japan": "Japão",
    "South Korea": "Coreia do Sul",
    "India": "Índia",
    "Mexico": "México",
    "Argentina": "Argentina",
    "Canada": "Canadá",
    "Australia": "Austrália",
    "Russia": "Rússia",
    "Netherlands": "Holanda",
    "Switzerland": "Suíça",
    "Belgium": "Bélgica",
    "Sweden": "Suécia",
    "Norway": "Noruega",
    "Denmark": "Dinamarca",
    "Austria": "Áustria",
    "Poland": "Polônia",
    "Greece": "Grécia",
    "Turkey": "Turquia",
    "Egypt": "Egito",
    "South Africa": "África do Sul",
    "New Zealand": "Nova Zelândia",
    "Chile": "Chile",
    "Colombia": "Colômbia",
    "Peru": "Peru",
    "Venezuela": "Venezuela",
    "Uruguay": "Uruguai",
    
    # Estados dos EUA (exemplos)
    "California": "Califórnia",
    "New York": "Nova York",
    "Florida": "Flórida",
    "Texas": "Texas",
    "Illinois": "Illinois",
    "Pennsylvania": "Pensilvânia",
    "Ohio": "Ohio",
    "Georgia": "Geórgia",
    "North Carolina": "Carolina do Norte",
    "Michigan": "Michigan",
    
    
    # Ocupações comuns
    "Engineer": "Engenheiro",
    "Doctor": "Médico",
    "Teacher": "Professor",
    "Nurse": "Enfermeiro",
    "Lawyer": "Advogado",
    "Accountant": "Contador",
    "Manager": "Gerente",
    "Developer": "Desenvolvedor",
    "Designer": "Designer",
    "Analyst": "Analista",
    "Consultant": "Consultor",
    "Director": "Diretor",
    "Executive": "Executivo",
    "Administrator": "Administrador",
    "Technician": "Técnico",
    "Specialist": "Especialista",
    "Coordinator": "Coordenador",
    "Supervisor": "Supervisor",
    "Assistant": "Assistente",
    "Sales": "Vendas",
    "Marketing": "Marketing",
    "Student": "Estudante",
    "Retired": "Aposentado",
    "Self-employed": "Autônomo",
    "Unemployed": "Desempregado",
    "Entrepreneur": "Empreendedor",
    "Freelancer": "Freelancer",
}

def translate_batch(texts, target_lang='pt-br'):
    """Traduz múltiplos textos com traduções personalizadas"""
    if not texts:
        return {}
    
    # Separar textos que já têm tradução customizada
    custom_results = {}
    texts_to_translate = []
    
    for text in texts:
        if text in CUSTOM_TRANSLATIONS:
            custom_results[text] = CUSTOM_TRANSLATIONS[text]
            print(f"  ✓ Tradução customizada: {text} → {CUSTOM_TRANSLATIONS[text]}")
        else:
            texts_to_translate.append(text)
    
    # Se não há nada para traduzir via API, retornar só os customizados
    if not texts_to_translate:
        return custom_results
    
    print(f"  🌐 Traduzindo via API: {len(texts_to_translate)} valores...")
    
    # Traduzir o restante via API (em lotes de 100)
    path = '/translate'
    constructed_url = TRANSLATOR_ENDPOINT + path
    
    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': target_lang
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': TRANSLATOR_LOCATION,
        'Content-type': 'application/json'
    }
    
    # Azure Translator aceita até 100 textos por requisição
    batch_size = 100
    for i in range(0, len(texts_to_translate), batch_size):
        batch = texts_to_translate[i:i+batch_size]
        body = [{'text': str(text)} for text in batch]
        
        try:
            response = requests.post(constructed_url, params=params, headers=headers, json=body)
            response.raise_for_status()
            results = response.json()
            
            # Adicionar traduções da API
            for j, result in enumerate(results):
                original = batch[j]
                translated = result['translations'][0]['text']
                custom_results[original] = translated
                
        except Exception as e:
            print(f"  ❌ Erro na tradução do lote {i//batch_size + 1}: {e}")
            # Em caso de erro, manter texto original
            for text in batch:
                if text not in custom_results:
                    custom_results[text] = text
    
    return custom_results

print("="*70)
print("🚀 INICIANDO TRADUÇÃO E CRIAÇÃO DA TABELA SILVER_CUSTOMER")
print("="*70)

# Ler tabela staging
print("\n📖 Lendo tabela stg_customer...")
df_stg = spark.table("stg_customer")
total_rows = df_stg.count()
print(f"   Total de registros: {total_rows:,}")

# Mostrar schema original
print("\n📋 Schema da tabela original:")
df_stg.printSchema()

# Coletar valores únicos de cada coluna
print("\n📊 Coletando valores únicos para tradução...")

print("   - Gender...")
unique_genders = [row.Gender for row in df_stg.select("Gender").distinct().collect() if row.Gender]
print(f"     {len(unique_genders)} valores únicos")

print("   - Occupation...")
unique_occupations = [row.Occupation for row in df_stg.select("Occupation").distinct().collect() if row.Occupation]
print(f"     {len(unique_occupations)} valores únicos")

print("   - StateFull...")
unique_states = [row.StateFull for row in df_stg.select("StateFull").distinct().collect() if row.StateFull]
print(f"     {len(unique_states)} valores únicos")

print("   - CountryFull...")
unique_countries = [row.CountryFull for row in df_stg.select("CountryFull").distinct().collect() if row.CountryFull]
print(f"     {len(unique_countries)} valores únicos")

print("   - Continent...")
unique_continents = [row.Continent for row in df_stg.select("Continent").distinct().collect() if row.Continent]
print(f"     {len(unique_continents)} valores únicos")

# Traduzir valores únicos
print("\n🌐 Traduzindo valores únicos...")

print("\n1️⃣ Traduzindo GÊNEROS:")
gender_map = translate_batch(unique_genders)
print(f"   Resultado: {gender_map}")

print("\n2️⃣ Traduzindo CONTINENTES:")
continent_map = translate_batch(unique_continents)
print(f"   Resultado: {continent_map}")

print("\n3️⃣ Traduzindo PAÍSES:")
country_map = translate_batch(unique_countries)
print(f"   Primeiros 5: {dict(list(country_map.items())[:5])}")

print("\n4️⃣ Traduzindo ESTADOS:")
state_map = translate_batch(unique_states)
print(f"   Primeiros 5: {dict(list(state_map.items())[:5])}")

print("\n5️⃣ Traduzindo OCUPAÇÕES:")
occupation_map = translate_batch(unique_occupations)
print(f"   Primeiros 5: {dict(list(occupation_map.items())[:5])}")

# Criar UDFs para mapear traduções
print("\n🔧 Criando funções de mapeamento...")
gender_udf = udf(lambda x: gender_map.get(x, x) if x else None, StringType())
occupation_udf = udf(lambda x: occupation_map.get(x, x) if x else None, StringType())
state_udf = udf(lambda x: state_map.get(x, x) if x else None, StringType())
country_udf = udf(lambda x: country_map.get(x, x) if x else None, StringType())
continent_udf = udf(lambda x: continent_map.get(x, x) if x else None, StringType())

# Aplicar traduções e criar tabela silver
print("\n✨ Criando tabela silver_customer com colunas traduzidas...")
df_silver = df_stg \
    .withColumn("genero", gender_udf(col("Gender"))) \
    .withColumn("ocupacao", occupation_udf(col("Occupation"))) \
    .withColumn("estado", state_udf(col("StateFull"))) \
    .withColumn("pais", country_udf(col("CountryFull"))) \
    .withColumn("continente", continent_udf(col("Continent")))

# Reordenar colunas (português depois do inglês)
print("\n📐 Reordenando colunas...")
# Pegar todas as colunas originais
original_cols = df_stg.columns

# Criar lista de colunas na ordem desejada
ordered_cols = []
for col_name in original_cols:
    ordered_cols.append(col_name)
    # Adicionar coluna em português logo após a inglesa
    if col_name == "Gender":
        ordered_cols.append("genero")
    elif col_name == "Occupation":
        ordered_cols.append("ocupacao")
    elif col_name == "StateFull":
        ordered_cols.append("estado")
    elif col_name == "CountryFull":
        ordered_cols.append("pais")
    elif col_name == "Continent":
        ordered_cols.append("continente")

df_silver = df_silver.select(ordered_cols)

# Mostrar preview
print("\n👁️ Preview da tabela silver_customer:")
df_silver.select(
    "Gender", "genero",
    "Occupation", "ocupacao",
    "CountryFull", "pais", 
    "Continent", "continente"
).show(10, truncate=False)

# Salvar tabela silver
print("\n💾 Salvando tabela silver_customer...")
df_silver.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("silver_customer")

# Verificar criação
print("\n✅ Tabela silver_customer criada com sucesso!")
row_count = spark.table("silver_customer").count()
print(f"   Total de registros: {row_count:,}")

print("\n📋 Schema da tabela silver_customer:")
spark.table("silver_customer").printSchema()

# Estatísticas finais
print("\n" + "="*70)
print("📊 RESUMO DA TRADUÇÃO")
print("="*70)
print(f"✓ Gêneros traduzidos: {len(gender_map)}")
print(f"✓ Ocupações traduzidas: {len(occupation_map)}")
print(f"✓ Estados traduzidos: {len(state_map)}")
print(f"✓ Países traduzidos: {len(country_map)}")
print(f"✓ Continentes traduzidos: {len(continent_map)}")
print(f"✓ Total de registros processados: {row_count:,}")
print("="*70)
print("🎉 PROCESSO CONCLUÍDO COM SUCESSO!")
print("="*70)

### Criar colunas traduzidas para português - tabela Date

###### Cria a tabela "silver_date" adicionando novas colunas com tradução em português (ano mês, ano mês reduzido, mês, mês reduzido, dia da semana e dia da semana reduzido)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import requests
import json

# ============= CONFIGURE AQUI =============
TRANSLATOR_KEY = "xxx"
TRANSLATOR_ENDPOINT = "https://api.cognitive.microsofttranslator.com"
TRANSLATOR_LOCATION = "xxx"
# ==========================================

# Dicionário de traduções personalizadas para DATAS
CUSTOM_TRANSLATIONS = {
    # Meses completos
    "January": "Janeiro",
    "February": "Fevereiro",
    "March": "Março",
    "April": "Abril",
    "May": "Maio",
    "June": "Junho",
    "July": "Julho",
    "August": "Agosto",
    "September": "Setembro",
    "October": "Outubro",
    "November": "Novembro",
    "December": "Dezembro",
    
    # Meses abreviados
    "Jan": "Jan",
    "Feb": "Fev",
    "Mar": "Mar",
    "Apr": "Abr",
    "May": "Mai",
    "Jun": "Jun",
    "Jul": "Jul",
    "Aug": "Ago",
    "Sep": "Set",
    "Oct": "Out",
    "Nov": "Nov",
    "Dec": "Dez",
    
    # Dias da semana completos
    "Monday": "Segunda-feira",
    "Tuesday": "Terça-feira",
    "Wednesday": "Quarta-feira",
    "Thursday": "Quinta-feira",
    "Friday": "Sexta-feira",
    "Saturday": "Sábado",
    "Sunday": "Domingo",
    
    # Dias da semana abreviados
    "Mon": "Seg",
    "Tue": "Ter",
    "Wed": "Qua",
    "Thu": "Qui",
    "Fri": "Sex",
    "Sat": "Sáb",
    "Sun": "Dom",
    
    # Combinações comuns de mês + ano (exemplos)
    "January 2015": "Janeiro 2015",
    "February 2015": "Fevereiro 2015",
    "March 2015": "Março 2015",
    "April 2015": "Abril 2015",
    "May 2015": "Maio 2015",
    "June 2015": "Junho 2015",
    "July 2015": "Julho 2015",
    "August 2015": "Agosto 2015",
    "September 2015": "Setembro 2015",
    "October 2015": "Outubro 2015",
    "November 2015": "Novembro 2015",
    "December 2015": "Dezembro 2015",
    
    # Abreviados + ano
    "Jan 2015": "Jan 2015",
    "Feb 2015": "Fev 2015",
    "Mar 2015": "Mar 2015",
    "Apr 2015": "Abr 2015",
    "May 2015": "Mai 2015",
    "Jun 2015": "Jun 2015",
    "Jul 2015": "Jul 2015",
    "Aug 2015": "Ago 2015",
    "Sep 2015": "Set 2015",
    "Oct 2015": "Out 2015",
    "Nov 2015": "Nov 2015",
    "Dec 2015": "Dez 2015",
}

def translate_date_text(text):
    """Traduz textos de datas substituindo partes conhecidas"""
    if not text or text == "":
        return text
    
    # Se já está no dicionário customizado, retornar diretamente
    if text in CUSTOM_TRANSLATIONS:
        return CUSTOM_TRANSLATIONS[text]
    
    # Tentar traduzir por partes (para lidar com diferentes anos)
    translated = text
    
    # Substituir meses completos
    for eng, pt in {
        "January": "Janeiro", "February": "Fevereiro", "March": "Março",
        "April": "Abril", "May": "Maio", "June": "Junho",
        "July": "Julho", "August": "Agosto", "September": "Setembro",
        "October": "Outubro", "November": "Novembro", "December": "Dezembro"
    }.items():
        if eng in translated:
            translated = translated.replace(eng, pt)
            return translated
    
    # Substituir meses abreviados
    for eng, pt in {
        "Jan": "Jan", "Feb": "Fev", "Mar": "Mar", "Apr": "Abr",
        "May": "Mai", "Jun": "Jun", "Jul": "Jul", "Aug": "Ago",
        "Sep": "Set", "Oct": "Out", "Nov": "Nov", "Dec": "Dez"
    }.items():
        if eng in translated:
            translated = translated.replace(eng, pt)
            return translated
    
    # Substituir dias da semana completos
    for eng, pt in {
        "Monday": "Segunda-feira", "Tuesday": "Terça-feira", 
        "Wednesday": "Quarta-feira", "Thursday": "Quinta-feira",
        "Friday": "Sexta-feira", "Saturday": "Sábado", "Sunday": "Domingo"
    }.items():
        if eng in translated:
            translated = translated.replace(eng, pt)
            return translated
    
    # Substituir dias da semana abreviados
    for eng, pt in {
        "Mon": "Seg", "Tue": "Ter", "Wed": "Qua", "Thu": "Qui",
        "Fri": "Sex", "Sat": "Sáb", "Sun": "Dom"
    }.items():
        if eng in translated:
            translated = translated.replace(eng, pt)
            return translated
    
    # Se não encontrou nada, retornar original
    return text

def translate_batch_dates(texts):
    """Traduz múltiplos textos de datas"""
    if not texts:
        return {}
    
    results = {}
    
    for text in texts:
        translated = translate_date_text(text)
        results[text] = translated
        if translated != text:
            print(f"  ✓ Traduzido: {text} → {translated}")
    
    return results

print("="*70)
print("🚀 INICIANDO TRADUÇÃO E CRIAÇÃO DA TABELA SILVER_DATE")
print("="*70)

# Ler tabela staging
print("\n📖 Lendo tabela stg_date...")
df_stg = spark.table("stg_date")
total_rows = df_stg.count()
print(f"   Total de registros: {total_rows:,}")

# Mostrar schema original
print("\n📋 Schema da tabela original:")
df_stg.printSchema()

# Coletar valores únicos de cada coluna
print("\n📊 Coletando valores únicos para tradução...")

print("   - YearMonth...")
unique_yearmonth = [row.YearMonth for row in df_stg.select("YearMonth").distinct().collect() if row.YearMonth]
print(f"     {len(unique_yearmonth)} valores únicos")

print("   - YearMonthShort...")
unique_yearmonthshort = [row.YearMonthShort for row in df_stg.select("YearMonthShort").distinct().collect() if row.YearMonthShort]
print(f"     {len(unique_yearmonthshort)} valores únicos")

print("   - Month...")
unique_month = [row.Month for row in df_stg.select("Month").distinct().collect() if row.Month]
print(f"     {len(unique_month)} valores únicos")

print("   - MonthShort...")
unique_monthshort = [row.MonthShort for row in df_stg.select("MonthShort").distinct().collect() if row.MonthShort]
print(f"     {len(unique_monthshort)} valores únicos")

print("   - DayofWeek...")
unique_dayofweek = [row.DayofWeek for row in df_stg.select("DayofWeek").distinct().collect() if row.DayofWeek]
print(f"     {len(unique_dayofweek)} valores únicos")

print("   - DayofWeekShort...")
unique_dayofweekshort = [row.DayofWeekShort for row in df_stg.select("DayofWeekShort").distinct().collect() if row.DayofWeekShort]
print(f"     {len(unique_dayofweekshort)} valores únicos")

# Traduzir valores únicos
print("\n🌐 Traduzindo valores únicos...")

print("\n1️⃣ Traduzindo ANO MÊS (YearMonth):")
yearmonth_map = translate_batch_dates(unique_yearmonth)
print(f"   Primeiros 5: {dict(list(yearmonth_map.items())[:5])}")

print("\n2️⃣ Traduzindo ANO MÊS REDUZIDO (YearMonthShort):")
yearmonthshort_map = translate_batch_dates(unique_yearmonthshort)
print(f"   Primeiros 5: {dict(list(yearmonthshort_map.items())[:5])}")

print("\n3️⃣ Traduzindo MÊS (Month):")
month_map = translate_batch_dates(unique_month)
print(f"   Resultado: {month_map}")

print("\n4️⃣ Traduzindo MÊS REDUZIDO (MonthShort):")
monthshort_map = translate_batch_dates(unique_monthshort)
print(f"   Resultado: {monthshort_map}")

print("\n5️⃣ Traduzindo DIA DA SEMANA (DayofWeek):")
dayofweek_map = translate_batch_dates(unique_dayofweek)
print(f"   Resultado: {dayofweek_map}")

print("\n6️⃣ Traduzindo DIA DA SEMANA REDUZIDO (DayofWeekShort):")
dayofweekshort_map = translate_batch_dates(unique_dayofweekshort)
print(f"   Resultado: {dayofweekshort_map}")

# Criar UDFs para mapear traduções
print("\n🔧 Criando funções de mapeamento...")
yearmonth_udf = udf(lambda x: yearmonth_map.get(x, x) if x else None, StringType())
yearmonthshort_udf = udf(lambda x: yearmonthshort_map.get(x, x) if x else None, StringType())
month_udf = udf(lambda x: month_map.get(x, x) if x else None, StringType())
monthshort_udf = udf(lambda x: monthshort_map.get(x, x) if x else None, StringType())
dayofweek_udf = udf(lambda x: dayofweek_map.get(x, x) if x else None, StringType())
dayofweekshort_udf = udf(lambda x: dayofweekshort_map.get(x, x) if x else None, StringType())

# Aplicar traduções e criar tabela silver
print("\n✨ Criando tabela silver_date com colunas traduzidas...")
df_silver = df_stg \
    .withColumn("ano_mes", yearmonth_udf(col("YearMonth"))) \
    .withColumn("ano_mes_reduzido", yearmonthshort_udf(col("YearMonthShort"))) \
    .withColumn("mes", month_udf(col("Month"))) \
    .withColumn("mes_reduzido", monthshort_udf(col("MonthShort"))) \
    .withColumn("dia_da_semana", dayofweek_udf(col("DayofWeek"))) \
    .withColumn("dia_da_semana_reduzido", dayofweekshort_udf(col("DayofWeekShort")))

# Reordenar colunas (português depois do inglês)
print("\n📐 Reordenando colunas...")
# Pegar todas as colunas originais
original_cols = df_stg.columns

# Criar lista de colunas na ordem desejada
ordered_cols = []
for col_name in original_cols:
    ordered_cols.append(col_name)
    # Adicionar coluna em português logo após a inglesa
    if col_name == "YearMonth":
        ordered_cols.append("ano_mes")
    elif col_name == "YearMonthShort":
        ordered_cols.append("ano_mes_reduzido")
    elif col_name == "Month":
        ordered_cols.append("mes")
    elif col_name == "MonthShort":
        ordered_cols.append("mes_reduzido")
    elif col_name == "DayofWeek":
        ordered_cols.append("dia_da_semana")
    elif col_name == "DayofWeekShort":
        ordered_cols.append("dia_da_semana_reduzido")

df_silver = df_silver.select(ordered_cols)

# Mostrar preview
print("\n👁️ Preview da tabela silver_date:")
df_silver.select(
    "YearMonth", "ano_mes",
    "Month", "mes",
    "DayofWeek", "dia_da_semana"
).show(10, truncate=False)

# Salvar tabela silver com overwriteSchema
print("\n💾 Salvando tabela silver_date...")
df_silver.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_date")

# Verificar criação
print("\n✅ Tabela silver_date criada com sucesso!")
row_count = spark.table("silver_date").count()
print(f"   Total de registros: {row_count:,}")

print("\n📋 Schema da tabela silver_date:")
spark.table("silver_date").printSchema()

# Estatísticas finais
print("\n" + "="*70)
print("📊 RESUMO DA TRADUÇÃO")
print("="*70)
print(f"✓ Ano Mês traduzidos: {len(yearmonth_map)}")
print(f"✓ Ano Mês Reduzido traduzidos: {len(yearmonthshort_map)}")
print(f"✓ Meses traduzidos: {len(month_map)}")
print(f"✓ Meses Reduzido traduzidos: {len(monthshort_map)}")
print(f"✓ Dias da Semana traduzidos: {len(dayofweek_map)}")
print(f"✓ Dias da Semana Reduzido traduzidos: {len(dayofweekshort_map)}")
print(f"✓ Total de registros processados: {row_count:,}")
print("="*70)
print("🎉 PROCESSO CONCLUÍDO COM SUCESSO!")
print("="*70)

### Criar colunas traduzidas para português - tabela Product

###### Cria a tabela "silver_product" adicionando novas colunas com tradução em português (nome_produto, cor, unidade_peso, subcategoria e categoria)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import requests
import json
import re

# ============= CONFIGURE AQUI =============
TRANSLATOR_KEY = "xxx"
TRANSLATOR_ENDPOINT = "https://api.cognitive.microsofttranslator.com"
TRANSLATOR_LOCATION = "xxx"
# ==========================================

# Dicionário de traduções personalizadas para PRODUTOS
CUSTOM_TRANSLATIONS = {
    # Cores comuns
    "Black": "Preto",
    "White": "Branco",
    "Red": "Vermelho",
    "Blue": "Azul",
    "blue": "Azul",
    "Green": "Verde",
    "Yellow": "Amarelo",
    "Orange": "Laranja",
    "Purple": "Roxo",
    "Pink": "Rosa",
    "Brown": "Marrom",
    "Gray": "Cinza",
    "Grey": "Cinza",
    "Silver": "Prata",
    "Gold": "Dourado",
    "Beige": "Bege",
    "Navy": "Azul Marinho",
    "Transparent": "Transparente",
    "Clear": "Transparente",
    
    # Unidades de peso
    "ounces": "onças",
    "ounce": "onça",
    "pounds": "libras",
    "pound": "libra",
    "grams": "gramas",
    "gram": "grama",
    "kilograms": "quilogramas",
    "kilogram": "quilograma",
    "kg": "kg",
    "g": "g",
    "oz": "oz",
    "lb": "lb",
    "lbs": "lbs",
    
    # Categorias e Subcategorias comuns de eletrônicos
    "Cameras and camcorders": "Câmeras e Filmadoras",
    "Cameras & camcorders": "Câmeras e Filmadoras",
    "Cameras": "Câmeras",
    "Camcorders": "Filmadoras",
    "Camera Accessories": "Acessórios para Câmeras",
    "Games and Toys": "Jogos e Brinquedos",
    "Audio": "Áudio",
    "Computers": "Computadores",
    "TV and Video": "TV e Vídeo",
    "Cell phones": "Celulares",
    "Cell Phones": "Celulares",
    "Home Appliances": "Eletrodomésticos",
    "Music, Movies and Audio Books": "Música, Filmes e Audiolivros",
    "Download Games":	"Jogos Baixados",
    "Bluetooth Headphones":	"Fones de Ouvido Bluetooth",
    "Televisions":	"Televisores",
    "Car Video":	"Vídeo Automotivo",
    "Digital Cameras":	"Câmeras Digitais",
    "Projectors & Screens":	"Projetores e Telas",
    "MP4&MP3":	"MP4 e MP3",
    "Touch Screen Phones": 	"Telefones com Tela Sensível ao Toque",
    "Desktops":	"Computadores de Mesa",
    "Monitors":	"Monitores",
    "Fans":	"Ventiladores",
    "Lamps":	"Lâmpadas",
    "Smart phones & PDAs": 	"Smartphones e PDAs",
    "Movie DVD":	"DVDs de Filmes",
    "Cameras & Camcorders Accessories":	"Acessórios para Câmeras e Filmadoras",
    "Water Heaters":	"Aquecedores de Água",
    "Refrigerators":	"Refrigeradores",
    "VCD & DVD":	"VCDs e DVDs",
    "Computers Accessories":	"Acessórios para Computadores",
    "Boxed Games":	"Jogos em Caixa",
    "Printers, Scanners & Fax":	"Impressoras, Scanners e Fax",
    "Home Theater System":	"Sistemas de Home Theater",
    "Recording Pen":	"Caneta Gravadora",
    "Washers & Dryers":	"Lavadoras e Secadoras",
    "Air Conditioners":	"Ar-condicionado",
    "Home & Office Phones":	"Telefones Residenciais e Comerciais",
    "Microwaves":	"Micro-ondas",
    "Digital SLR Cameras":	"Câmeras SLR Digitais",
    "Cell phones Accessories":	"Acessórios para celulares",
    "Coffee Machines":	"Máquinas de café",
    "Camcorders": "Filmadoras",

    
    # Palavras comuns em nomes de produtos
    "Telephoto": "Telefoto",
    "Lens": "Lente",
    "Battery": "Bateria",
    "Charger": "Carregador",
    "Case": "Estojo",
    "Cover": "Capa",
    "Screen": "Tela",
    "Protector": "Protetor",
    "Cable": "Cabo",
    "Adapter": "Adaptador",
    "Memory": "Memória",
    "Card": "Cartão",
    "Tripod": "Tripé",
    "Flash": "Flash",
    "Filter": "Filtro",
    "Bag": "Bolsa",
    "Mount": "Suporte",
    "Remote": "Controle Remoto",
    "Wireless": "Sem Fio",
    "Bluetooth": "Bluetooth",
    "USB": "USB",
    "HDMI": "HDMI",
}

# ============= SIGLAS/TERMOS A PRESERVAR =============
# Adicione aqui as siglas que NÃO devem ser traduzidas
PRESERVE_TERMS = [
    "WWI",
    "WWII",
    "USB",
    "HDMI",
    "DVD",
    "CD",
    "MP3",
    "HD",
    "4K",
    "8K",
    "LED",
    "LCD",
    "OLED",
    "Wi-Fi",
    "WiFi",
    "Bluetooth",
    "GPS",
    # Adicione outras siglas conforme necessário
]

def fix_translation_with_preserved_terms(original, translated, preserve_terms):
    """
    Corrige tradução restaurando termos que deveriam ser preservados.
    Exemplo: se "WWI" virou "Primeira Guerra Mundial", volta para "WWI"
    """
    result = translated
    
    for term in preserve_terms:
        if term in original and term not in result:
            # A sigla estava no original mas não está na tradução
            # Tentar identificar o que ela virou e substituir de volta
            
            # Casos específicos conhecidos
            replacements = {
                "WWI": ["Primeira Guerra Mundial", "primeira guerra mundial", "WWI", "Wwi"],
                "WWII": ["Segunda Guerra Mundial", "segunda guerra mundial", "WWII", "Wwii"],
                "USB": ["usb"],
                "HDMI": ["hdmi"],
                "HD": ["hd", "alta definição", "Alta Definição"],
                "DVD": ["dvd"],
                "CD": ["cd"],
                "Wi-Fi": ["wi-fi", "wifi", "WiFi"],
                "GPS": ["gps"],
            }
            
            if term in replacements:
                for wrong in replacements[term]:
                    result = result.replace(wrong, term)
    
    return result

def translate_batch(texts, target_lang='pt-br'):
    """Traduz múltiplos textos em lote (RÁPIDO)"""
    if not texts:
        return {}
    
    custom_results = {}
    texts_to_translate = []
    
    for text in texts:
        if text in CUSTOM_TRANSLATIONS:
            custom_results[text] = CUSTOM_TRANSLATIONS[text]
            print(f"  ✓ Tradução customizada: {text} → {CUSTOM_TRANSLATIONS[text]}")
        else:
            texts_to_translate.append(text)
    
    if not texts_to_translate:
        return custom_results
    
    print(f"  🌐 Traduzindo via API em lote: {len(texts_to_translate)} valores...")
    
    # Traduzir em lotes de 100 (MUITO MAIS RÁPIDO)
    batch_size = 100
    path = '/translate'
    constructed_url = TRANSLATOR_ENDPOINT + path
    
    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': target_lang
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': TRANSLATOR_LOCATION,
        'Content-type': 'application/json'
    }
    
    for i in range(0, len(texts_to_translate), batch_size):
        batch = texts_to_translate[i:i+batch_size]
        body = [{'text': str(text)} for text in batch]
        
        try:
            response = requests.post(constructed_url, params=params, headers=headers, json=body)
            response.raise_for_status()
            results = response.json()
            
            for j, result in enumerate(results):
                original = batch[j]
                translated = result['translations'][0]['text']
                
                # Corrigir tradução preservando termos especiais
                translated = fix_translation_with_preserved_terms(original, translated, PRESERVE_TERMS)
                
                custom_results[original] = translated
                
        except Exception as e:
            print(f"  ❌ Erro na tradução do lote {i//batch_size + 1}: {e}")
            for text in batch:
                if text not in custom_results:
                    custom_results[text] = text
    
    return custom_results

print("="*70)
print("🚀 INICIANDO TRADUÇÃO E CRIAÇÃO DA TABELA SILVER_PRODUCT")
print("="*70)

# Ler tabela staging
print("\n📖 Lendo tabela stg_product...")
df_stg = spark.table("stg_product")
total_rows = df_stg.count()
print(f"   Total de registros: {total_rows:,}")

# Mostrar schema original
print("\n📋 Schema da tabela original:")
df_stg.printSchema()

# Coletar valores únicos de cada coluna
print("\n📊 Coletando valores únicos para tradução...")

print("   - ProductName...")
unique_productname = [row.ProductName for row in df_stg.select("ProductName").distinct().collect() if row.ProductName]
print(f"     {len(unique_productname)} valores únicos")

print("   - Color...")
unique_color = [row.Color for row in df_stg.select("Color").distinct().collect() if row.Color]
print(f"     {len(unique_color)} valores únicos")

print("   - WeightUnit...")
unique_weightunit = [row.WeightUnit for row in df_stg.select("WeightUnit").distinct().collect() if row.WeightUnit]
print(f"     {len(unique_weightunit)} valores únicos")

print("   - SubCategoryName...")
unique_subcategory = [row.SubCategoryName for row in df_stg.select("SubCategoryName").distinct().collect() if row.SubCategoryName]
print(f"     {len(unique_subcategory)} valores únicos")

print("   - CategoryName...")
unique_category = [row.CategoryName for row in df_stg.select("CategoryName").distinct().collect() if row.CategoryName]
print(f"     {len(unique_category)} valores únicos")

# Traduzir valores únicos
print("\n🌐 Traduzindo valores únicos...")

print("\n1️⃣ Traduzindo NOMES DE PRODUTOS (ProductName):")
productname_map = translate_batch(unique_productname)  # TRADUÇÃO EM LOTE - RÁPIDA!
print(f"   Total traduzido: {len(productname_map)}")
print(f"   Primeiros 5 exemplos:")
for i, (orig, trans) in enumerate(list(productname_map.items())[:5]):
    print(f"     {orig} → {trans}")

print("\n2️⃣ Traduzindo CORES (Color):")
color_map = translate_batch(unique_color)
print(f"   Resultado: {color_map}")

print("\n3️⃣ Traduzindo UNIDADES DE PESO (WeightUnit):")
weightunit_map = translate_batch(unique_weightunit)
print(f"   Resultado: {weightunit_map}")

print("\n4️⃣ Traduzindo SUBCATEGORIAS (SubCategoryName):")
subcategory_map = translate_batch(unique_subcategory)
print(f"   Total traduzido: {len(subcategory_map)}")

print("\n5️⃣ Traduzindo CATEGORIAS (CategoryName):")
category_map = translate_batch(unique_category)
print(f"   Resultado: {category_map}")

# Criar UDFs para mapear traduções
print("\n🔧 Criando funções de mapeamento...")
productname_udf = udf(lambda x: productname_map.get(x, x) if x else None, StringType())
color_udf = udf(lambda x: color_map.get(x, x) if x else None, StringType())
weightunit_udf = udf(lambda x: weightunit_map.get(x, x) if x else None, StringType())
subcategory_udf = udf(lambda x: subcategory_map.get(x, x) if x else None, StringType())
category_udf = udf(lambda x: category_map.get(x, x) if x else None, StringType())

# Aplicar traduções e criar tabela silver
print("\n✨ Criando tabela silver_product com colunas traduzidas...")
df_silver = df_stg \
    .withColumn("nome_produto", productname_udf(col("ProductName"))) \
    .withColumn("cor", color_udf(col("Color"))) \
    .withColumn("unidade_de_peso", weightunit_udf(col("WeightUnit"))) \
    .withColumn("subcategoria", subcategory_udf(col("SubCategoryName"))) \
    .withColumn("categoria", category_udf(col("CategoryName")))

# Reordenar colunas (português depois do inglês)
print("\n📐 Reordenando colunas...")
original_cols = df_stg.columns

ordered_cols = []
for col_name in original_cols:
    ordered_cols.append(col_name)
    if col_name == "ProductName":
        ordered_cols.append("nome_produto")
    elif col_name == "Color":
        ordered_cols.append("cor")
    elif col_name == "WeightUnit":
        ordered_cols.append("unidade_de_peso")
    elif col_name == "SubCategoryName":
        ordered_cols.append("subcategoria")
    elif col_name == "CategoryName":
        ordered_cols.append("categoria")

df_silver = df_silver.select(ordered_cols)

# Mostrar preview
print("\n👁️ Preview da tabela silver_product:")
df_silver.select(
    "ProductName", "nome_produto",
    "Color", "cor",
    "CategoryName", "categoria"
).show(10, truncate=False)

# Salvar tabela silver com overwriteSchema
print("\n💾 Salvando tabela silver_product...")
df_silver.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_product")

# Verificar criação
print("\n✅ Tabela silver_product criada com sucesso!")
row_count = spark.table("silver_product").count()
print(f"   Total de registros: {row_count:,}")

print("\n📋 Schema da tabela silver_product:")
spark.table("silver_product").printSchema()

# Estatísticas finais
print("\n" + "="*70)
print("📊 RESUMO DA TRADUÇÃO")
print("="*70)
print(f"✓ Nomes de produtos traduzidos: {len(productname_map)}")
print(f"✓ Cores traduzidas: {len(color_map)}")
print(f"✓ Unidades de peso traduzidas: {len(weightunit_map)}")
print(f"✓ Subcategorias traduzidas: {len(subcategory_map)}")
print(f"✓ Categorias traduzidas: {len(category_map)}")
print(f"✓ Total de registros processados: {row_count:,}")
print("="*70)
print("🎉 PROCESSO CONCLUÍDO COM SUCESSO!")
print("="*70)

### Criar colunas traduzidas para português - tabela Store

###### Cria a tabela "silver_store" adicionando novas colunas com tradução em português (nome_pais, estado e situação)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import requests
import json

# ============= CONFIGURE AQUI =============
TRANSLATOR_KEY = "xxx"
TRANSLATOR_ENDPOINT = "https://api.cognitive.microsofttranslator.com"
TRANSLATOR_LOCATION = "xxx"
# ==========================================

# Dicionário de traduções personalizadas para LOJAS
CUSTOM_TRANSLATIONS = {
    # Países
    "United States": "Estados Unidos",
    "United Kingdom": "Reino Unido",
    "Brazil": "Brasil",
    "Germany": "Alemanha",
    "France": "França",
    "Spain": "Espanha",
    "Italy": "Itália",
    "Portugal": "Portugal",
    "China": "China",
    "Japan": "Japão",
    "South Korea": "Coreia do Sul",
    "India": "Índia",
    "Mexico": "México",
    "Argentina": "Argentina",
    "Canada": "Canadá",
    "Australia": "Austrália",
    "Russia": "Rússia",
    "Netherlands": "Holanda",
    "Switzerland": "Suíça",
    "Belgium": "Bélgica",
    "Sweden": "Suécia",
    "Norway": "Noruega",
    "Denmark": "Dinamarca",
    "Austria": "Áustria",
    "Poland": "Polônia",
    "Greece": "Grécia",
    "Turkey": "Turquia",
    "Egypt": "Egito",
    "South Africa": "África do Sul",
    "New Zealand": "Nova Zelândia",
    "Chile": "Chile",
    "Colombia": "Colômbia",
    "Peru": "Peru",
    "Venezuela": "Venezuela",
    "Uruguay": "Uruguai",
    
    # Estados dos EUA
    "Alabama": "Alabama",
    "Alaska": "Alasca",
    "Arizona": "Arizona",
    "Arkansas": "Arkansas",
    "California": "Califórnia",
    "Colorado": "Colorado",
    "Connecticut": "Connecticut",
    "Delaware": "Delaware",
    "Florida": "Flórida",
    "Georgia": "Geórgia",
    "Hawaii": "Havaí",
    "Idaho": "Idaho",
    "Illinois": "Illinois",
    "Indiana": "Indiana",
    "Iowa": "Iowa",
    "Kansas": "Kansas",
    "Kentucky": "Kentucky",
    "Louisiana": "Louisiana",
    "Maine": "Maine",
    "Maryland": "Maryland",
    "Massachusetts": "Massachusetts",
    "Michigan": "Michigan",
    "Minnesota": "Minnesota",
    "Mississippi": "Mississippi",
    "Missouri": "Missouri",
    "Montana": "Montana",
    "Nebraska": "Nebraska",
    "Nevada": "Nevada",
    "New Hampshire": "New Hampshire",
    "New Jersey": "Nova Jersey",
    "New Mexico": "Novo México",
    "New México": "Novo México",  # Corrigindo possível erro de digitação
    "New York": "Nova York",
    "North Carolina": "Carolina do Norte",
    "North Dakota": "Dakota do Norte",
    "Ohio": "Ohio",
    "Oklahoma": "Oklahoma",
    "Oregon": "Oregon",
    "Pennsylvania": "Pensilvânia",
    "Rhode Island": "Rhode Island",
    "South Carolina": "Carolina do Sul",
    "South Dakota": "Dakota do Sul",
    "Tennessee": "Tennessee",
    "Texas": "Texas",
    "Utah": "Utah",
    "Vermont": "Vermont",
    "Virginia": "Virgínia",
    "Washington": "Washington",
    "West Virginia": "Virgínia Ocidental",
    "Wisconsin": "Wisconsin",
    "Wyoming": "Wyoming",
    
    # Províncias do Canadá
    "Ontario": "Ontário",
    "Quebec": "Quebec",
    "British Columbia": "Colúmbia Britânica",
    "Alberta": "Alberta",
    "Manitoba": "Manitoba",
    "Saskatchewan": "Saskatchewan",
    "Nova Scotia": "Nova Escócia",
    "New Brunswick": "Nova Brunswick",
    "Newfoundland and Labrador": "Terra Nova e Labrador",
    "Prince Edward Island": "Ilha do Príncipe Eduardo",
    "Northwest Territories": "Territórios do Noroeste",
    "Yukon": "Yukon",
    "Nunavut": "Nunavut",
    
    # Status de lojas
    "Active": "Ativo",
    "Inactive": "Inativo",
    "Open": "Aberto",
    "Closed": "Fechado",
    "Restructured": "Reestruturado",
    "Restructuring": "Em Reestruturação",
    "Under Construction": "Em Construção",
    "Planned": "Planejado",
    "Temporary Closed": "Temporariamente Fechado",
    "Permanently Closed": "Fechado Permanentemente",
    "Relocated": "Relocado",
    "Renovating": "Em Renovação",
    "Opening Soon": "Abrindo em Breve",
}

def translate_batch(texts, target_lang='pt-br'):
    """Traduz múltiplos textos em lote (RÁPIDO)"""
    if not texts:
        return {}
    
    custom_results = {}
    texts_to_translate = []
    
    for text in texts:
        if text in CUSTOM_TRANSLATIONS:
            custom_results[text] = CUSTOM_TRANSLATIONS[text]
            print(f"  ✓ Tradução customizada: {text} → {CUSTOM_TRANSLATIONS[text]}")
        else:
            texts_to_translate.append(text)
    
    if not texts_to_translate:
        return custom_results
    
    print(f"  🌐 Traduzindo via API em lote: {len(texts_to_translate)} valores...")
    
    # Traduzir em lotes de 100
    batch_size = 100
    path = '/translate'
    constructed_url = TRANSLATOR_ENDPOINT + path
    
    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': target_lang
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': TRANSLATOR_LOCATION,
        'Content-type': 'application/json'
    }
    
    for i in range(0, len(texts_to_translate), batch_size):
        batch = texts_to_translate[i:i+batch_size]
        body = [{'text': str(text)} for text in batch]
        
        try:
            response = requests.post(constructed_url, params=params, headers=headers, json=body)
            response.raise_for_status()
            results = response.json()
            
            for j, result in enumerate(results):
                original = batch[j]
                translated = result['translations'][0]['text']
                custom_results[original] = translated
                
        except Exception as e:
            print(f"  ❌ Erro na tradução do lote {i//batch_size + 1}: {e}")
            for text in batch:
                if text not in custom_results:
                    custom_results[text] = text
    
    return custom_results

print("="*70)
print("🚀 INICIANDO TRADUÇÃO E CRIAÇÃO DA TABELA SILVER_STORE")
print("="*70)

# Ler tabela staging
print("\n📖 Lendo tabela stg_store...")
df_stg = spark.table("stg_store")
total_rows = df_stg.count()
print(f"   Total de registros: {total_rows:,}")

# Mostrar schema original
print("\n📋 Schema da tabela original:")
df_stg.printSchema()

# Coletar valores únicos de cada coluna
print("\n📊 Coletando valores únicos para tradução...")

print("   - CountryName...")
unique_country = [row.CountryName for row in df_stg.select("CountryName").distinct().collect() if row.CountryName]
print(f"     {len(unique_country)} valores únicos")

print("   - State...")
unique_state = [row.State for row in df_stg.select("State").distinct().collect() if row.State]
print(f"     {len(unique_state)} valores únicos")

print("   - Status...")
unique_status = [row.Status for row in df_stg.select("Status").distinct().collect() if row.Status]
print(f"     {len(unique_status)} valores únicos")

# Traduzir valores únicos
print("\n🌐 Traduzindo valores únicos...")

print("\n1️⃣ Traduzindo PAÍSES (CountryName):")
country_map = translate_batch(unique_country)
print(f"   Resultado: {country_map}")

print("\n2️⃣ Traduzindo ESTADOS (State):")
state_map = translate_batch(unique_state)
print(f"   Total traduzido: {len(state_map)}")
print(f"   Primeiros 5 exemplos:")
for i, (orig, trans) in enumerate(list(state_map.items())[:5]):
    print(f"     {orig} → {trans}")

print("\n3️⃣ Traduzindo STATUS (Status):")
status_map = translate_batch(unique_status)
print(f"   Resultado: {status_map}")

# Criar UDFs para mapear traduções
print("\n🔧 Criando funções de mapeamento...")
country_udf = udf(lambda x: country_map.get(x, x) if x else None, StringType())
state_udf = udf(lambda x: state_map.get(x, x) if x else None, StringType())
status_udf = udf(lambda x: status_map.get(x, x) if x else None, StringType())

# Aplicar traduções e criar tabela silver
print("\n✨ Criando tabela silver_store com colunas traduzidas...")
df_silver = df_stg \
    .withColumn("nome_pais", country_udf(col("CountryName"))) \
    .withColumn("estado", state_udf(col("State"))) \
    .withColumn("situacao", status_udf(col("Status")))

# Reordenar colunas (português depois do inglês)
print("\n📐 Reordenando colunas...")
original_cols = df_stg.columns

ordered_cols = []
for col_name in original_cols:
    ordered_cols.append(col_name)
    if col_name == "CountryName":
        ordered_cols.append("nome_pais")
    elif col_name == "State":
        ordered_cols.append("estado")
    elif col_name == "Status":
        ordered_cols.append("situacao")

df_silver = df_silver.select(ordered_cols)

# Mostrar preview
print("\n👁️ Preview da tabela silver_store:")
df_silver.select(
    "CountryName", "nome_pais",
    "State", "estado",
    "Status", "situacao"
).show(10, truncate=False)

# Salvar tabela silver com overwriteSchema
print("\n💾 Salvando tabela silver_store...")
df_silver.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_store")

# Verificar criação
print("\n✅ Tabela silver_store criada com sucesso!")
row_count = spark.table("silver_store").count()
print(f"   Total de registros: {row_count:,}")

print("\n📋 Schema da tabela silver_store:")
spark.table("silver_store").printSchema()

# Estatísticas finais
print("\n" + "="*70)
print("📊 RESUMO DA TRADUÇÃO")
print("="*70)
print(f"✓ Países traduzidos: {len(country_map)}")
print(f"✓ Estados traduzidos: {len(state_map)}")
print(f"✓ Status traduzidos: {len(status_map)}")
print(f"✓ Total de registros processados: {row_count:,}")
print("="*70)
print("🎉 PROCESSO CONCLUÍDO COM SUCESSO!")
print("="*70)

#### Criando tabela fato "Sales"

###### Cria tabela "silver_sales" no lakehouse sem transformações adicionais

In [ ]:
df_sales = spark.sql("SELECT * FROM LH_Contoso.stg_sales")
# Salvar tabela silver
print("\n💾 Salvando tabela silver_sales...")
df_sales.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("silver_sales")

# Verificar criação
print("\n✅ Tabela silver_sales criada com sucesso!")
row_count = spark.table("silver_sales").count()
print(f"   Total de registros: {row_count:,}")